# Data transformation

In [42]:
import pandas as pd
from numpy import mean
#import os
from functools import reduce

In [3]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

## Load the data

In [6]:
characters_g = pd.read_pickle('chatacter_data/genderized/characters_genderized.pckl')
#characters_g

In [7]:
movies = pd.read_pickle('movies_data/movies_df.pckl')
#movies

## Clean the data

In [8]:
# remove all movies that do not have a year
movies_clean = movies.loc[pd.to_numeric(movies['year'], errors='coerce').notnull()]
#souce: https://www.py4u.net/discuss/13840

In [9]:
print(f"{round((len(movies)-len(movies_clean))/len(movies)*100)}% of movies have no year.")

2% of movies have no year.


In [10]:
# remove all characters where the gender is not known
characters_clean = characters_g.loc[logical_not(logical_or(characters_g['gender']=='unknown' ,  
                                                           characters_g['gender'].isnull()))]
print(f"{round((len(characters_g)-len(characters_clean))/len(characters_g)*100)}% of characters have no gender.")

25% of characters have no gender.


In [70]:
movies_clean.to_pickle('movies_data/movies_clean_df.pckl')
characters_clean.to_pickle('chatacter_data/genderized/characters_genderized_clean_df.pckl')

## Transform the data

In [11]:
movies = pd.read_pickle("movies_data/movies_clean_df.pckl")
characters = pd.read_pickle('chatacter_data/genderized/characters_genderized_clean_df.pckl')

**Goals**

I'm looking for something like this:

````python
plot_data_number_of_characters_m_f = []
plot_data_occurence_of_characters_m_f = []
plot_data_character_word_count_m_f = []

analysis = ["movie_id",
            "movie_year",
            "character_count_total",
            "character_percentage_m",
            "character_percentage_f", 
            "character_occourence_total",
            "character_occourence_m_percentage",
            "character_occourence_f_percentage",
            "word_count_total",
            "word_percentage_dialouge_m",
            "word_percentage_dialouge_f"]
````

***
### character counts

In [17]:
movie_character_count = characters[['movie_id','character_id']]\
                        .groupby(['movie_id'])\
                        .size().reset_index(name='character_count')
movie_character_count.head()

,movie_id,character_count
0,100000.0,8
1,100001.0,2
2,100002.0,9
3,100003.0,6
4,100004.0,7


In [18]:
movie_f_counts = characters[['movie_id','character_id', 'gender']]\
                    .loc[characters['gender']=='female']\
                    .groupby(['movie_id', 'gender'])\
                    .size().reset_index(name='female_count')\
                    [['movie_id', 'female_count']]

movie_f_counts.head()

,movie_id,female_count
0,100000.0,3
1,100002.0,3
2,100004.0,1
3,100005.0,3
4,100006.0,1


In [19]:
movie_m_counts = characters[['movie_id','character_id', 'gender']]\
                    .loc[characters['gender']=='male']\
                    .groupby(['movie_id', 'gender'])\
                    .size().reset_index(name='male_count')\
                    [['movie_id', 'male_count']]
movie_m_counts.head()

,movie_id,male_count
0,100000.0,5
1,100001.0,2
2,100002.0,6
3,100003.0,6
4,100004.0,6


***
### word counts

In [20]:
movie_word_counts_total = characters[['movie_id','word_count']]\
                            .groupby(['movie_id'])\
                            .sum().reset_index()
movie_word_counts_total.head()

,movie_id,word_count
0,100000.0,7482.0
1,100001.0,2.0
2,100002.0,5867.0
3,100003.0,10329.0
4,100004.0,6233.0


In [21]:
movie_word_counts_m = characters[['movie_id', 'gender', 'word_count']]\
                    .loc[characters['gender']=='male']\
                    .groupby(['movie_id', 'gender'])\
                    .sum().reset_index()\
                    [['movie_id','word_count']]\
                    .rename({'word_count':'word_count_m'}, axis='columns')

movie_word_counts_m.head() 

,movie_id,word_count_m
0,100000.0,3995.0
1,100001.0,2.0
2,100002.0,3919.0
3,100003.0,10329.0
4,100004.0,5889.0


In [22]:
movie_word_counts_f = characters[['movie_id', 'gender', 'word_count']]\
                    .loc[characters['gender']=='female']\
                    .groupby(['movie_id', 'gender'])\
                    .sum().reset_index()\
                    [['movie_id','word_count']]\
                    .rename({'word_count':'word_count_f'}, axis='columns')
movie_word_counts_f.head() 

,movie_id,word_count_f
0,100000.0,3487.0
1,100002.0,1948.0
2,100004.0,344.0
3,100005.0,517.0
4,100006.0,357.0


***
### character occourences

In [23]:
character_occourences_total = characters[['movie_id', 'occourences']]\
                    .groupby(['movie_id'])\
                    .sum().reset_index()\
                    [['movie_id','occourences']]\
                    .rename({'occourences':'characters_occourences'}, axis='columns')
character_occourences_total.head() 

,movie_id,characters_occourences
0,100000.0,882.0
1,100001.0,2.0
2,100002.0,592.0
3,100003.0,502.0
4,100004.0,389.0


In [24]:
character_occourences_f = characters[['movie_id','gender','occourences']]\
                    .loc[characters['gender']=='female']\
                    .groupby(['movie_id','gender'])\
                    .sum().reset_index()\
                    [['movie_id','occourences']]\
                    .rename({'occourences':'characters_occourences_f'}, axis='columns')
character_occourences_f.head() 

,movie_id,characters_occourences_f
0,100000.0,389.0
1,100002.0,189.0
2,100004.0,22.0
3,100005.0,49.0
4,100006.0,31.0


In [25]:
character_occourences_m = characters[['movie_id','gender','occourences']]\
                    .loc[characters['gender']=='male']\
                    .groupby(['movie_id','gender'])\
                    .sum().reset_index()\
                    [['movie_id','occourences']]\
                    .rename({'occourences':'characters_occourences_m'}, axis='columns')
character_occourences_m.head() 

,movie_id,characters_occourences_m
0,100000.0,493.0
1,100001.0,2.0
2,100002.0,403.0
3,100003.0,502.0
4,100004.0,367.0


***
### Combine tables and create calculated columns

In [26]:
analysis_content = [movies[['movie_id','year']], 
                    movie_character_count,
                    movie_m_counts, 
                    movie_f_counts, 
                    movie_word_counts_total,
                    movie_word_counts_m,
                    movie_word_counts_f,
                    character_occourences_total,
                    character_occourences_f,
                    character_occourences_m]
analysis = reduce(lambda left,right: pd.merge(left,right,on='movie_id'), analysis_content)
#source: https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0


In [27]:
# add male character percenrage
analysis['male_character_percentage'] = round((analysis['male_count']/analysis['character_count'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71


In [28]:
# add female character percentage
analysis['female_character_percentage'] = round((analysis['female_count']/analysis['character_count'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage,female_character_percentage
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50,37.50
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67,33.33
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71,14.29
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00,60.00
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71,14.29


In [29]:
# add word count percentage female
analysis['word_percentage_f'] = round((analysis['word_count_f']/analysis['word_count'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage,female_character_percentage,word_percentage_f
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50,37.50,46.61
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67,33.33,33.20
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71,14.29,5.52
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00,60.00,15.55
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71,14.29,4.72


In [30]:
# add word count percentage male
analysis['word_percentage_m'] = round((analysis['word_count_m']/analysis['word_count'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage,female_character_percentage,word_percentage_f,word_percentage_m
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50,37.50,46.61,53.39
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67,33.33,33.20,66.80
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71,14.29,5.52,94.48
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00,60.00,15.55,84.45
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71,14.29,4.72,95.28


In [31]:
# add percentage female occourences
analysis['character_occourence_percentage_f'] = round((analysis['characters_occourences_f']/analysis['characters_occourences'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage,female_character_percentage,word_percentage_f,word_percentage_m,character_occourence_percentage_f
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50,37.50,46.61,53.39,44.10
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67,33.33,33.20,66.80,31.93
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71,14.29,5.52,94.48,5.66
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00,60.00,15.55,84.45,32.45
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71,14.29,4.72,95.28,7.23


In [32]:
# add percentage male occourences
analysis['character_occourence_percentage_m'] = round((analysis['characters_occourences_m']/analysis['characters_occourences'])*100,2)
analysis.head()

,movie_id,year,character_count,male_count,female_count,word_count,word_count_m,word_count_f,characters_occourences,characters_occourences_f,characters_occourences_m,male_character_percentage,female_character_percentage,word_percentage_f,word_percentage_m,character_occourence_percentage_f,character_occourence_percentage_m
0,100000,1997,8,5,3,7482.0,3995.0,3487.0,882.0,389.0,493.0,62.50,37.50,46.61,53.39,44.10,55.90
1,100002,2004,9,6,3,5867.0,3919.0,1948.0,592.0,189.0,403.0,66.67,33.33,33.20,66.80,31.93,68.07
2,100004,2013,7,6,1,6233.0,5889.0,344.0,389.0,22.0,367.0,85.71,14.29,5.52,94.48,5.66,94.34
3,100005,2010,5,2,3,3324.0,2807.0,517.0,151.0,49.0,102.0,40.00,60.00,15.55,84.45,32.45,67.55
4,100006,1991,7,6,1,7556.0,7199.0,357.0,429.0,31.0,398.0,85.71,14.29,4.72,95.28,7.23,92.77


In [33]:
#analysis.to_pickle('analysis_data/analysis.pckl')

***
### Create the plot input

In [44]:
analysis.columns

Index(['movie_id', 'year', 'character_count', 'male_count', 'female_count',
       'word_count', 'word_count_m', 'word_count_f', 'characters_occourences',
       'characters_occourences_f', 'characters_occourences_m',
       'male_character_percentage', 'female_character_percentage',
       'word_percentage_f', 'word_percentage_m',
       'character_occourence_percentage_f',
       'character_occourence_percentage_m'],
      dtype='object')

In [49]:
#source: https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/53096340
character_count_data = analysis[['year','movie_id','male_character_percentage', 'female_character_percentage']]
character_count_data_groupped=character_count_data.groupby(['year']).agg({ 'movie_id':list,
                                            'male_character_percentage':mean,
                                           'female_character_percentage':mean})
character_count_data_groupped

,movie_id,male_character_percentage,female_character_percentage
year,,,
1933,[100364],83.330000,16.670000
1939,"[100817, 101192]",50.000000,50.000000
1940,"[100494, 100549]",72.225000,27.775000
1941,[100266],71.430000,28.570000
1943,[100102],42.860000,57.140000
1944,"[100357, 101073]",69.050000,30.950000
1946,[100607],66.670000,33.330000
1947,[101041],87.500000,12.500000
1948,[100779],57.140000,42.860000


In [65]:
character_occurrence_data = analysis[['year','movie_id','character_occourence_percentage_m', 'character_occourence_percentage_f']]
character_occurrence_data_groupped=character_occurrence_data.groupby(['year']).agg({ 'movie_id':list,
                                            'character_occourence_percentage_m':mean,
                                           'character_occourence_percentage_f':mean})
character_occurrence_data_groupped

,movie_id,character_occourence_percentage_m,character_occourence_percentage_f
year,,,
1933,[100364],95.280000,4.720000
1939,"[100817, 101192]",44.085000,55.915000
1940,"[100494, 100549]",66.345000,33.655000
1941,[100266],81.020000,18.980000
1943,[100102],57.450000,42.550000
1944,"[100357, 101073]",73.905000,26.095000
1946,[100607],82.460000,17.540000
1947,[101041],97.240000,2.760000
1948,[100779],65.010000,34.990000


In [67]:
character_word_count_data = analysis[['year','movie_id','word_percentage_m', 'word_percentage_f']]
character_word_count_data_groupped=character_word_count_data.groupby(['year']).agg({ 'movie_id':list,
                                            'word_percentage_m':mean,
                                           'word_percentage_f':mean})
character_word_count_data_groupped

,movie_id,word_percentage_m,word_percentage_f
year,,,
1933,[100364],97.370000,2.630000
1939,"[100817, 101192]",42.820000,57.180000
1940,"[100494, 100549]",62.795000,37.205000
1941,[100266],81.560000,18.440000
1943,[100102],62.030000,37.970000
1944,"[100357, 101073]",77.060000,22.940000
1946,[100607],86.810000,13.190000
1947,[101041],99.940000,0.060000
1948,[100779],71.600000,28.400000


In [68]:
character_word_count_data_groupped.to_pickle('analysis_data/character_word_count_data_groupped')
character_occurrence_data_groupped.to_pickle('analysis_data/character_occurrence_data_groupped')
character_count_data_groupped.to_pickle('analysis_data/character_count_data_groupped')